In [7]:
import os
import openai
import json
from dotenv import load_dotenv

In [2]:
load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

In [3]:
def chat_completion()-> str:
    completion : openai.ChatCompletion = openai.ChatCompletion.create(
        model = 'gpt-3.5-turbo-1106',
        messages = [
            {'role': 'system', 'content': 'You are a poetic assistant, skilled in explaining complex programming concepts in creative poetry'},
            {'role': 'user', 'content':'Compose a poem that explains the concept of recursion in programming'}
        ]
    )
    return completion.choices[0].message.content

print(chat_completion())

In the land of code, a technique so noble,
A pattern of repetition, known as recursion.
A function calling itself, a tale retold,
In a loop of elegance, a story's evolution.

Like a mirror reflecting its own reflection,
Recursion calls upon itself without hesitation.
Breaking down problems, with grace and finesse,
Weaving through layers, with code's caress.

In the realm of algorithms, it holds great might,
Solving puzzles, unfolding with each recursive sight.
A dance of iterations, a beautiful art,
Unfolding patterns, written in the heart.

But beware, dear coder, of infinite recurrence,
A looping nightmare, a dreaded occurrence.
For in the world of recursion, balance is key,
To unlock its wonders, with mindful decree.

So embrace the power, but wield it with care,
Let recursion guide you, through the code's affair.
A poetic dance, in the programmer's hand,
Recursion, a marvel, in the digital land.


We can make the model return output in json format


In [9]:
# even the output is in the json format, the type of output is string
# because the output of llm is always string
response = openai.ChatCompletion.create(
    model = 'gpt-3.5-turbo-1106',
    response_format={'type': 'json_object'},
    messages=[
            {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
            {"role": "user", "content": "List of months that have 30 days"}
            ]
)

print(response.choices[0].message.content)
print(type(response.choices[0].message.content))

{
  "months": ["April", "June", "September", "November"]
}
<class 'str'>


In [8]:
# even if you dont specify response format, it still will be string
json_data = openai.ChatCompletion.create(
    model = 'gpt-3.5-turbo-1106',
    #response_format = {'type':'json_object'},
    messages = [
        {'role':'system', "content": "You are a helpful assistant designed to output JSON format"},
        {'role':'user', "content": "What are the advantages to convert the output of ChatGPT into JSON format"}
    ]
)

print(json_data.choices[0].message.content)
print(type(json_data.choices[0].message.content))

Converting the output of ChatGPT into JSON format offers several advantages:

1. Structured data: JSON format provides a structured and easy-to-read representation of the output, making it more convenient for parsing and further processing.

2. Interoperability: JSON is widely supported and can be easily integrated with a variety of programming languages and applications, making it easier to exchange information between different systems.

3. Customization: JSON format allows for the inclusion of key-value pairs, which can be customized to include specific metadata or additional information in the output.

4. Standardization: Using JSON format helps standardize the output format, making it easier for developers to understand and work with the output across different applications or use cases.

Overall, converting the output of ChatGPT into JSON format can improve the ease of use, flexibility, and interoperability of the data.
<class 'str'>


## Function Calling


In [14]:
#custom function
import json

def get_current_weather(location: str, unit: str = 'fahrenheit')-> str:
    """Get the current weather in a given location"""
    if 'tokyo' in location.lower():
        return json.dumps({'location':'Tokyo', 
                        'temperature': '10',
                        'unit': 'celsius'})
    elif 'san francisco' in location.lower():
        return json.dumps({'location':'San francisco', 
                    'temperature': '72',
                    'unit': 'fahrenheit'})
    elif 'paris' in location.lower():
        return json.dumps({'location':'Paris', 
                        'temperature': '22',
                        'unit': 'celsius'})
    else:
        return json.dumps({'location':location, 
                        'temperature': 'unknown'})
        

In [15]:
def run_conversation(main_request: str)->str:
    #Step 1: Send the conversation and available functions to the model
    
    messages = [{'role': 'user','content':main_request}] #user messages list
    
    #along with the prompt messages, we give details about the function
    
    tools=[#list of function to be passed along the prompt
        { #first function to be passed along the prompt
            'type':'function',# define what you are passing. Here it is function
            'function':{ #what the function is
                'name': 'get_current_weather', #name of the function
                'description': 'Get the current weather in a give location',
                #what the function is doing (important for NLP to understand 
                # what the function is used for)
                'parameters': { #what is passed as argument to function
                    'type': 'object',
                    'properties':{
                        'location': {#first parameter
                            'type': 'string', #type of location parameter
                            'description': 'The city and state e.g San Francisco, CA',
                            #description of the parameter
                        },
                        'unit': { #second parameter
                            'type':'string', #type of the unit parameter
                            'enum': ['celsius', 'fahrenheit']
                        }
                    },
                    'required': ['location'] #required parameter (if not 
                    #provided, the function will not run)
                }
            }
        }
    ]
    
    # First request
    
    response: openai.ChatCompletion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-1106", #model selection
        messages=messages, #message list
        tools=tools, #list of dictionary to be passed
        tool_choice="auto",  # automatically select function that is to be used
        #if general quesiton asked, it will respond with it's own knowledge
        #but if asked particular question, it will auto call function
    )
    
    response_message = response.choices[0].message
    display("* First Response: ", dict(response_message))

    
    tool_calls = response_message.tool_calls #additional parameter
    display("First Response Tool Calls: ", tool_calls) #give details about
    #list of function that is to be called
    
    #Step 2: Check if the model wanted to call a function
    
    if tool_calls: #if the list have any element then what to do
        #Step 3: Call the function. The JSON response may not always be correct
        #Be sure to handle errors
        
        available_functions = {
            "get_current_weather": get_current_weather,
        } #
        
        messages.append(response_message)  
        # extend conversation with assistant's reply
        
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name #extract the name of function
            
            function_to_call = available_functions[function_name] # check if the
            # present in available_functions. If so, assign the function name
            # to the variable
            
            function_args = json.loads(tool_call.function.arguments) #all the 
            #parameter of the function is converted into JSON (dictionary)
            
            function_response = function_to_call( 
                location=function_args.get("location"),#get location parameter
                unit=function_args.get("unit"), #get unit parameter
            )
            
            messages.append( #response after running custom function is appended
                            #to the messages list (thread)
                {
                    "tool_call_id": tool_call.id, #which function is running and
                    #its id
                    "role": "tool", #role of custom function (tool)
                    "name": function_name, #name of custom function 
                    "content": function_response, #response of the custom function
                }
            )  # extend conversation with function response
            
        display("* Second Request Messages: ", list(messages)) 
        
        second_response: openai.ChatCompletion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-1106",
            messages=messages,
        )  # get a new response from the model where it can see the function 
        #response
        
        print("* Second Response: ", dict(second_response))
        
        return second_response.choices[0].message.content
    


In [16]:
run_conversation("What's the weather like in San Francisco, Tokyo, and Paris?")

'* First Response: '

{'role': 'assistant',
 'content': None,
 'tool_calls': [<OpenAIObject id=call_9mVXmG2Tq7gygHKiHrrNgGMj at 0x2761eb786d0> JSON: {
    "id": "call_9mVXmG2Tq7gygHKiHrrNgGMj",
    "type": "function",
    "function": {
      "name": "get_current_weather",
      "arguments": "{\"location\": \"San Francisco\", \"unit\": \"celsius\"}"
    }
  },
  <OpenAIObject id=call_d3HL4bCS3SgZWRotzMoLW3FD at 0x2761eb78c20> JSON: {
    "id": "call_d3HL4bCS3SgZWRotzMoLW3FD",
    "type": "function",
    "function": {
      "name": "get_current_weather",
      "arguments": "{\"location\": \"Tokyo\", \"unit\": \"celsius\"}"
    }
  },
  <OpenAIObject id=call_hC0KZrJb67WIRwiHUblepU5t at 0x2761ea13740> JSON: {
    "id": "call_hC0KZrJb67WIRwiHUblepU5t",
    "type": "function",
    "function": {
      "name": "get_current_weather",
      "arguments": "{\"location\": \"Paris\", \"unit\": \"celsius\"}"
    }
  }]}

'First Response Tool Calls: '

[<OpenAIObject id=call_9mVXmG2Tq7gygHKiHrrNgGMj at 0x2761eb786d0> JSON: {
   "id": "call_9mVXmG2Tq7gygHKiHrrNgGMj",
   "type": "function",
   "function": {
     "name": "get_current_weather",
     "arguments": "{\"location\": \"San Francisco\", \"unit\": \"celsius\"}"
   }
 },
 <OpenAIObject id=call_d3HL4bCS3SgZWRotzMoLW3FD at 0x2761eb78c20> JSON: {
   "id": "call_d3HL4bCS3SgZWRotzMoLW3FD",
   "type": "function",
   "function": {
     "name": "get_current_weather",
     "arguments": "{\"location\": \"Tokyo\", \"unit\": \"celsius\"}"
   }
 },
 <OpenAIObject id=call_hC0KZrJb67WIRwiHUblepU5t at 0x2761ea13740> JSON: {
   "id": "call_hC0KZrJb67WIRwiHUblepU5t",
   "type": "function",
   "function": {
     "name": "get_current_weather",
     "arguments": "{\"location\": \"Paris\", \"unit\": \"celsius\"}"
   }
 }]

'* Second Request Messages: '

[{'role': 'user',
  'content': "What's the weather like in San Francisco, Tokyo, and Paris?"},
 <OpenAIObject at 0x2761eb78b30> JSON: {
   "role": "assistant",
   "content": null,
   "tool_calls": [
     {
       "id": "call_9mVXmG2Tq7gygHKiHrrNgGMj",
       "type": "function",
       "function": {
         "name": "get_current_weather",
         "arguments": "{\"location\": \"San Francisco\", \"unit\": \"celsius\"}"
       }
     },
     {
       "id": "call_d3HL4bCS3SgZWRotzMoLW3FD",
       "type": "function",
       "function": {
         "name": "get_current_weather",
         "arguments": "{\"location\": \"Tokyo\", \"unit\": \"celsius\"}"
       }
     },
     {
       "id": "call_hC0KZrJb67WIRwiHUblepU5t",
       "type": "function",
       "function": {
         "name": "get_current_weather",
         "arguments": "{\"location\": \"Paris\", \"unit\": \"celsius\"}"
       }
     }
   ]
 },
 {'tool_call_id': 'call_9mVXmG2Tq7gygHKiHrrNgGMj',
  'role': 'tool',
  'name': 'get_current_

* Second Response:  {'id': 'chatcmpl-8kmP6H0tdxqfeBVK5o0Bh7fZgs7Cw', 'object': 'chat.completion', 'created': 1706159844, 'model': 'gpt-3.5-turbo-1106', 'choices': [<OpenAIObject at 0x2761ea12a70> JSON: {
  "index": 0,
  "message": {
    "role": "assistant",
    "content": "The current weather in San Francisco is 72\u00b0F and sunny, in Tokyo it is 10\u00b0C and partly cloudy, and in Paris it is 22\u00b0C and mostly sunny."
  },
  "logprobs": null,
  "finish_reason": "stop"
}], 'usage': <OpenAIObject at 0x2761ea129d0> JSON: {
  "prompt_tokens": 170,
  "completion_tokens": 36,
  "total_tokens": 206
}, 'system_fingerprint': 'fp_aaa20cc2ba'}


'The current weather in San Francisco is 72°F and sunny, in Tokyo it is 10°C and partly cloudy, and in Paris it is 22°C and mostly sunny.'

In [3]:
def get_cryptocurrency_price(crypto: str)-> str:
    '''Get crypto current price'''
    if 'bitcoin' in crypto.lower():
        return json.dumps({'crypto': 'Bitcoin', 'price': '40,016'})
    elif 'ethereum'in crypto.lower():
        return json.dumps({'crypto': 'Ethereum', 'price': '2,222'})
    elif 'usdt' in crypto.lower():
        return json.dumps({'crypto': 'USDT', 'price': '0.999'})
    elif 'bnb' in crypto.lower():
        return json.dumps({'crypto': 'BNB', 'price': '290.33'})
    else:
        return json.dumps({'crypto': crypto, 'price':'unknown'})
    

In [22]:
def get_price_conversation(main_request: str)->str:
    
    message_list = [{'role':'user', 'content':main_request}]
    tools=[
        {
            'type':'function',
            'function':{
                'name':'get_cryptocurrency_price',
                'description': 'Get current price of crypto currency',
                'parameters':{
                    'type': 'object',
                    'properties':{
                        'crypto':{
                            'type': 'string',
                            'description':'Name of the crypto currency to get current price'
                        }
                    },
                    'required': ['crypto']
                }
            }
        }
    ]
    
    
    response = openai.ChatCompletion.create(
        model = 'gpt-3.5-turbo-1106',
        messages = message_list,
        tools = tools,
        tool_choice = 'auto'
    )
    
    assistant_message = response.choices[0].message
    tool_calls = assistant_message.tool_calls
    
    if tool_calls:
        available_functions = {
            'get_cryptocurrency_price': get_cryptocurrency_price,
        }
        
        message_list.append(assistant_message)
        
        for tool_call in tool_calls:
            
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            
            function_response = function_to_call(
                crypto = function_args.get('crypto')
            )
            
            message_list.append({
                'tool_call_id': tool_call.id,
                'role':'tool',
                'name': function_name,
                'content': function_response
            })
        
        second_response = openai.ChatCompletion.create(
            model = 'gpt-3.5-turbo-1106',
            messages = message_list,
        )
        
        return second_response.choices[0].message.content

In [23]:
get_price_conversation('What is the price of Ethereum today')

'The price of Ethereum today is $2,222.'

In [26]:
get_price_conversation('What is the price of Bitcoin today')

'The price of Bitcoin today is $40,016.'

In [31]:
get_price_conversation('What is the price of PKR today')

"I'm sorry, but I couldn't find the current price of PKR."

In [30]:
get_price_conversation('What the R&B music genre means?')

AttributeError: tool_calls